# Code

## 0. Imports

In [ ]:
%load_ext autoreload
%autoreload all

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

## 1. Gradientenverfahren 1D

### 1.0 Algorithmus

#### 1.0.1 Define function and the derivative

In [ ]:
# Funktion definieren
def fn1d(x: float) -> float:
    return x**5 - 3 * x**3 + x**2 + 2


# Händisch Ableitung bestimmen
def fn1d_prime(x: float) -> float:
    return 5 * x**4 - 9 * x**2 + 2 * x

#### 1.0.2 Gradient Descent block

In [ ]:
# Lokales Minimum finden mit Gradientenverfahren
# Parameters
x_start_d1 = 2.3  # Startpunkt
xmin = -(np.fabs(x_start_d1) + 1.0)  # borders of the axis
xmax = np.fabs(x_start_d1) + 1.0
lr = 1e-3  # Schrittgröße (aka. Learning Rate) -> smaller values lead to an increase in accuracy
significant_gradient = 1e-3
ys = []
xs = []

while True:  # loop as long as there is a significant on average
    grad = fn1d_prime(x_start_d1)
    if grad >= significant_gradient:
        ys.append(fn1d(x_start_d1))
        xs.append(x_start_d1)
        x_start_d1 = x_start_d1 - lr * grad  # optimize
    else:
        break

### 1.1 Plots

#### 1.1.1 Plot the Function

In [ ]:
# define range and values
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]
y_deriv = [fn1d_prime(val) for val in x]

fig = go.Figure(
    data=[
        go.Scatter(x=x, y=y, mode="lines"),
        go.Scatter(x=x, y=y_deriv, mode="lines"),
    ]
)

fig.update_layout(width=1200, height=800)

fig.show()

#### 1.1.2 Animate the process of Gradient descent 

In [ ]:
# define the range and the values
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]

# initialize the plot and the markers for gradient descent
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1200,
    height=800,
    xaxis=dict(range=(np.min(x) - 1, np.max(x) + 1), autorange=False, zeroline=False),
    yaxis=dict(range=(np.min(y) - 1, np.max(y) + 1), autorange=False, zeroline=False),
    title_text="2D Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 50, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 10, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()

## 2. Gradientenverfahren 2D

### 2.0 Algorithmus

#### 2.0.1 Define function and derivative

In [ ]:
# Funktion definieren
def fn2d_old(x: float, y: float) -> float:
    return np.sin(np.pow(x, 2) + np.pow(y, 2))


def fn2d_prime_old(x: float, y: float) -> np.ndarray:  # gibt 2D Array zurück
    partial_x = (
        2 * x * np.cos(np.pow(x, 2) + np.pow(y, 2))
    )  # leite f nach x ab, lass y konstant
    partial_y = (
        2 * y * np.cos(np.pow(x, 2) + np.pow(y, 2))
    )  # leite f nach y ab, lass x konstant
    return np.array([[partial_x], [partial_y]], ndmin=2)

$\dot{f}_{2d} = 0$ für $x = y = 0$

#### 2.0.2 Gradient Descent block


In [ ]:
# Lokales Minimum finden mit Gradientenverfahren
# Parameters
x_start = 0.1  # Startpunkt
y_start = -2.7
# define axis length of the plot
xy_min = np.negative((np.fabs(x_start) + np.fabs(y_start) + 1.0))
xy_max = np.fabs(x_start) + np.fabs(y_start) + 1.0
lr = 1e-2  # Schrittgröße
significant_gradient_2d = 1e-3
xs_2d = [x_start]
ys_2d = [y_start]
zs_2d = [fn2d_old(x_start, y_start)]

# same algorithm as in 1 dimensional gradient
while True:
    grad_xy = fn2d_prime_old(x_start, y_start)
    if np.mean(np.fabs(grad_xy)) >= significant_gradient_2d:
        x_start = x_start - lr * grad_xy[0, 0]
        y_start = y_start - lr * grad_xy[1, 0]
        zs_2d.append(fn2d_old(x_start, y_start))
        xs_2d.append(x_start)
        ys_2d.append(y_start)
    else:
        break

print(x_start, y_start)

### 2.1 Plots

#### 2.1.1 Plot the Surface

In [ ]:
# pass this argument to make sure the output is rendering in your notebook
pio.renderers.default = "notebook"

In [ ]:
# define range and meshgrid to let numpy handle the iteration through the arrays
r2 = np.arange(xy_min, xy_max, 0.1)
x2, y2 = np.meshgrid(r2, r2)

# calculate the corresponding values of the function
vals = fn2d_old(x2, y2)

# initialize the figure of the plot with the data
fig2 = go.Figure(
    data=[
        go.Surface(
            z=vals,
            x=x2,
            y=y2,
            colorscale="ice",
            showscale=False,
        )
    ]
)

# update layout
fig2.update_layout(
    title="Surface Plot Funktion",
    scene=dict(
        xaxis=dict(
            range=(np.min(x2) - 1, np.max(x2) + 1), autorange=False, zeroline=False
        ),
        yaxis=dict(
            range=(np.min(y2) - 1, np.max(y2) + 1), autorange=False, zeroline=False
        ),
        zaxis=dict(
            range=(np.min(vals) - 0.5, np.max(vals) + 0.5),
            autorange=False,
            zeroline=False,
        ),
    ),
    autosize=False,
    width=1200,
    height=600,
)

# show result
fig2.show()

#### 2.1.2 Plot the process of Gradient Descent

In [ ]:
# define range, meshgrid and values
r = np.arange(xy_min, xy_max, 0.1)
x, y = np.meshgrid(r, r)

vals_animation = fn2d_old(x, y)

# initialize figure with Surface plot and markers
fig = go.Figure(
    data=[
        go.Surface(
            z=vals_animation,
            x=x,
            y=y,
            opacity=0.5,
            colorscale="ice",
            name="Function Plot",
            showscale=False,
        ),
        go.Scatter3d(
            x=[xs_2d[0]],
            y=[ys_2d[0]],
            z=[zs_2d[0]],
            mode="markers",
            name="Current descent",
            marker=dict(color="darkred", size=5),
        ),
    ],
)

# configure layout
fig.update_layout(
    # define range of the different axes
    scene=dict(
        xaxis=dict(
            range=(np.min(x) - 0.5, np.max(x) + 0.5), autorange=False, zeroline=False
        ),
        yaxis=dict(
            range=(np.min(y) - 0.5, np.max(y) + 0.5), autorange=False, zeroline=False
        ),
        zaxis=dict(
            range=(np.min(vals_animation) - 0.5, np.max(vals_animation) + 0.5),
            autorange=False,
            zeroline=False,
        ),
    ),
    width=1200,
    height=800,
    title_text="3D Gradient Descent Animation",
    # animation configuration -> redraw needs to be true
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 50, "redraw": True},
                            "mode": "immediate",
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# handle animation frames
fig.update(
    frames=[
        go.Frame(
            data=[go.Scatter3d(x=[xs_2d[k]], y=[ys_2d[k]], z=[zs_2d[k]])], traces=[1]
        )
        for k in range(len(zs_2d))
    ]
)

# output the figure
fig.show()

### 2.2 Komponentenlose Variante

#### 2.2.1 Funktions- und Ableitungsdefinition

In [ ]:
# Funktion definieren
def fn2d(x: np.ndarray) -> np.float64:  # x of shape (2, 1)

    return np.sin(np.sum(x**2))


def fn2d_prime(x: np.ndarray) -> np.ndarray:  # gibt 2D Array zurück

    f_prime = 2 * x * np.cos(np.sum(x**2))

    return f_prime

#### 2.2.2 Gradient descent Prozess

In [ ]:
# Lokales Minimum finden mit Gradientenverfahren
# Parameters
xy_start = np.array(([0.1], [-2.7]), ndmin=2)  # Startpunkt

# define axis length of the plot
xy_min = np.negative((np.fabs(xy_start[0]) + np.fabs(xy_start[1]) + 1.0))
xy_max = np.fabs(xy_start[0]) + np.fabs(xy_start[1]) + 1.0
lr = 1e-2  # Schrittgröße
significant_gradient_2d = 1e-3

# same algorithm as in 1 dimensional gradient
while True:
    grad_xy = fn2d_prime(xy_start)
    if np.mean(np.fabs(grad_xy)) >= significant_gradient_2d:
        xy_start = xy_start - lr * grad_xy
    else:
        break

print(xy_start)

In [ ]:
# define range and meshgrid to let numpy handle the iteration through the arrays
r2 = np.arange(xy_min, xy_max, 0.1)
x2 = np.meshgrid(r2, r2)

# calculate the corresponding values of the function
vals = fn2d(x2)

# initialize the figure of the plot with the data
fig2 = go.Figure(
    data=[
        go.Surface(
            z=vals,
            x=x2[0],
            y=x2[1],
            colorscale="ice",
            showscale=False,
        )
    ]
)

# update layout
fig2.update_layout(
    title="Surface Plot Funktion",
    scene=dict(
        xaxis=dict(
            range=(np.min(x2[0]) - 1, np.max(x2[0]) + 1),
            autorange=False,
            zeroline=False,
        ),
        yaxis=dict(
            range=(np.min(x2[1]) - 1, np.max(x2[1]) + 1),
            autorange=False,
            zeroline=False,
        ),
        zaxis=dict(
            range=(np.min(vals) - 0.5, np.max(vals) + 0.5),
            autorange=False,
            zeroline=False,
        ),
    ),
    autosize=False,
    width=1200,
    height=600,
)

# show result
fig2.show()

## 3. Gradientenverfahren 3D

### 3.1 Define function and derivative

In [ ]:
# TODO: Komponentenschreibweise ersetzen, Dimensionalität der einzelnen Arrays durchgehen
def fn3d_old(x: float, y: float, z: float) -> float:

    return np.sin(np.exp(x) + np.exp(y) - np.cos(z))


def fn3d_prime_old(x: float, y: float, z: float) -> np.ndarray:
    partial_x = np.exp(x) * np.cos(np.exp(x) + np.exp(y) - np.cos(z))
    partial_y = np.exp(y) * np.cos(np.exp(x) + np.exp(y) - np.cos(z))
    partial_z = np.sin(z) * np.cos(np.exp(x) + np.exp(y) - np.cos(z))

    return np.array([[partial_x], [partial_y], [partial_z]], ndmin=2)

### 3.2 Gradient descent process

In [ ]:
# Parameters
x3d = 0.1  # Startpunkt
y3d = -1.7
z3d = 1.0
lr = 1e-1  # Schrittgröße
significant_gradient_3d = 1e-3
i = 0
iters3d = []
values3d = [fn3d_old(x3d, y3d, z3d)]

while True:
    grad_xyz = fn3d_prime_old(x3d, y3d, z3d)
    if np.mean(np.fabs(grad_xyz)) >= significant_gradient_3d:
        x3d = x3d - (lr * grad_xyz[0, 0])
        y3d = y3d - (lr * grad_xyz[1, 0])
        z3d = z3d - (lr * grad_xyz[2, 0])
        iters3d.append(i + 1) if i % 10 == 0 else None
        values3d.append(fn3d_old(x3d, y3d, z3d)) if i % 10 == 0 else None

        # Print process
        (
            print(f"Iteration: {i+1}, Funktionswert: {fn3d_old(x3d, y3d, z3d)}")
            if i % 1000 == 0
            else None
        )
    else:
        break
    i += 1

iters3d.append(i + 1)
values3d.append(fn3d_old(x3d, y3d, z3d))
print(f"Letzte Iteration: {i+1}, finaler Funktionswert: {fn3d_old(x3d, y3d, z3d)}")

### 3.3 Plot Funktionswerte

In [ ]:
# TODO: x-Achse logarithmisch plotten
fig = go.Figure(data=[go.Scatter(x=iters3d, y=values3d, mode="markers")])

fig.update_layout(width=1200, height=800, xaxis_type="log")

fig.show()

### 3.4 Komponentenlose Implementierung

In [ ]:
# TODO: Komponentenschreibweise ersetzen, Dimensionalität der einzelnen Arrays durchgehen
def fn3d(x: np.ndarray) -> float:  # x of shape (3, 1)

    return np.sin(np.exp(x[0]) + np.exp(x[1]) - np.cos(x[2]))


def fn3d_prime(x: np.ndarray) -> np.ndarray:
    partial_x = np.exp(x[0]) * np.cos(np.exp(x[0]) + np.exp(x[1]) - np.cos(x[2]))
    partial_y = np.exp(x[1]) * np.cos(np.exp(x[0]) + np.exp(x[1]) - np.cos(x[2]))
    partial_z = np.sin(x[2]) * np.cos(np.exp(x[0]) + np.exp(x[1]) - np.cos(x[2]))

    return np.array((partial_x, partial_y, partial_z), ndmin=2)

In [ ]:
# Parameters
start_xyz = np.array(([0.1], [-1.7], [1.0]), ndmin=2)  # Startpunkt
lr = 1e-1  # Schrittgröße
significant_gradient_3d = 1e-3
i = 0

while True:
    grad_xyz = fn3d_prime(start_xyz)
    if np.mean(np.fabs(grad_xyz)) >= significant_gradient_3d:
        start_xyz = start_xyz - (lr * grad_xyz)

        # Print process
        (
            print(f"Iteration: {i+1}, Funktionswert: {fn3d(start_xyz)}")
            if i % 100 == 0
            else None
        )
    else:
        break
    i += 1

print(f"Letzte Iteration: {i+1}, finaler Funktionswert: {fn3d(start_xyz)}")

## 4. Gradientenverfahren 4D

### 4.1 Define function and derivative

In [ ]:
def fn4d_old(x: float, y: float, z: float, w: float) -> float:

    return np.exp(np.sin(x) + np.cos(y) - np.pow(z, 2) + np.exp(w))


def fn4d_prime_old(x: float, y: float, z: float, w: float) -> np.ndarray:
    partial_x = np.cos(x) * np.exp(np.sin(x) + np.cos(y) - np.pow(z, 2) + np.exp(w))
    partial_y = np.sin(y) * np.exp(np.sin(x) + np.cos(y) - np.pow(z, 2) + np.exp(w))
    partial_z = -2 * z * np.exp(np.sin(x) + np.cos(y) - np.pow(z, 2) + np.exp(w))
    partial_w = np.exp(np.sin(x) + np.cos(y) - np.pow(z, 2) + np.exp(w))

    return np.array([[partial_x], [partial_y], [partial_z], [partial_w]], ndmin=2)

### 4.2 Gradient descent process

In [ ]:
# Parameters
x4d = 0.1  # Startpunkt
y4d = -2.7
z4d = 1.0
w4d = -1.0
lr = 1e-2  # Schrittgröße
significant_gradient_4d = 1e-2
# num_rep_2d = 10
change_4d = True
i = 0
iters4d = []
values4d = [fn4d_old(x4d, y4d, z4d, w4d)]

while change_4d is True:
    grad_xyzw = fn4d_prime_old(x4d, y4d, z4d, w4d)
    if np.mean(np.fabs(grad_xyzw)) >= significant_gradient_4d:
        iters4d.append(i + 1) if i % 10 == 0 else None
        values4d.append(fn4d_old(x4d, y4d, z4d, w4d)) if i % 10 == 0 else None
        x4d = x4d - lr * grad_xyzw[0, 0]
        y4d = y4d - lr * grad_xyzw[1, 0]
        z4d = z4d - lr * grad_xyzw[2, 0]
        w4d = w4d - lr * grad_xyzw[3, 0]
        (
            print(f"Iteraton: {i+1}, Funktionswert: {fn4d_old(x4d, y4d, z4d, w4d)}")
            if i % 100 == 0
            else None
        )
    else:
        change_4d = False
    i += 1
print(f"Letzte Iteration: {i+1}, finaler Funktionswert: {fn4d_old(x4d, y4d, z4d, w4d)}")
iters4d.append(i + 1)
values4d.append(fn4d_old(x4d, y4d, z4d, w4d))

### 4.3 Plot Funktionswerte

In [ ]:
fig = go.Figure(data=[go.Scatter(x=iters4d, y=values4d, mode="markers")])

fig.update_layout(width=1200, height=800)

fig.show()

### 4.4 Komponentenlose Vaiante

In [ ]:
def fn4d(x: np.ndarray) -> float:  # x of shape (4, 1)

    return np.exp(np.sin(x[0]) + np.cos(x[1]) - np.pow(x[2], 2) + np.exp(x[3]))


def fn4d_prime(x: np.ndarray) -> np.ndarray:
    partial_x = np.cos(x[0]) * np.exp(
        np.sin(x[0]) + np.cos(x[1]) - np.pow(x[2], 2) + np.exp(x[3])
    )
    partial_y = np.sin(x[1]) * np.exp(
        np.sin(x[0]) + np.cos(x[1]) - np.pow(x[2], 2) + np.exp(x[3])
    )
    partial_z = (
        -2 * x[2] * np.exp(np.sin(x[0]) + np.cos(x[1]) - np.pow(x[2], 2) + np.exp(x[3]))
    )
    partial_w = np.exp(np.sin(x[0]) + np.cos(x[1]) - np.pow(x[2], 2) + np.exp(x[3]))

    return np.array((partial_x, partial_y, partial_z, partial_w), ndmin=2)

In [ ]:
# Parameters
start_xyzw = np.array(([0.1], [-2.7], [1.0], [-1.0]), ndmin=2)  # Startpunkt

lr = 1e-2  # Schrittgröße
significant_gradient_4d = 1e-2
i = 1

while True:
    grad_xyzw = fn4d_prime(start_xyzw)
    if np.mean(np.fabs(grad_xyzw)) >= significant_gradient_4d:

        start_xyzw = start_xyzw - lr * grad_xyzw
        (
            print(f"Iteraton: {i}, Funktionswert: {fn4d(start_xyzw)}")
            if i % 100 == 0
            else None
        )
    else:
        break
    i += 1
print(f"Letzte Iteration: {i}, finaler Funktionswert: {fn4d(start_xyzw)}")